In [5]:
import os
import docx
import re
import csv
import pandas as pd
from fuzzywuzzy import fuzz
from collections import defaultdict
from difflib import SequenceMatcher, get_close_matches


<h3> Transcripts to CSVs </h3>

In [6]:

def clean_subtitle(text):
    # Remove asterisks and text between brackets
    text = re.sub(r'\*|\[.*?\]', '', text)
    text = re.sub(r'[\u2012\u2013\u2014\u2015]', '-', text)
    text = text.replace(";",":")
    return text.strip()

def convert_language_to_csv(language, input_file, subtitles):
    doc = docx.Document(input_file)
    subtitle_block = ""
    current_time_range = None
    current_highlight_color = None
    
    speaker=speakerDict[input_file.split("/")[-1].split(" (")[0].lower()]
    
    for paragraph in doc.paragraphs:
        text = paragraph.text.strip()
        text = clean_subtitle(text)  # Clean the subtitle text
        # Check if the paragraph contains a time range
        time_range_match = re.match(r'^(\d+[:;]\d+(?::\d+){0,2}\s*-\s*\d+[:;]\d+(?::\d+){0,2})\s*\**', text)
        if time_range_match:
            time_range = time_range_match.group(1)
            # If there's a previous subtitle block, add it to the dictionary
            if subtitle_block and current_time_range:
                subtitles[current_time_range][language] = {
                    "text": subtitle_block,
                    "highlight_color": current_highlight_color,
                    "comments": "",  # Initialize an empty "Comments" column
                    "speaker":speaker
                }
            subtitle_block = ""
            current_time_range = time_range
            current_highlight_color = None  # Reset the highlight color for a new time range
        else:
            # Extract the highlight color for each run in the paragraph
            for run in paragraph.runs:
                if run.font.highlight_color is not None:
                    current_highlight_color = run.font.highlight_color
            subtitle_block += text + "\n"

    # Add the last subtitle block to the dictionary
    if subtitle_block and current_time_range:
        subtitles[current_time_range][language] = {
            "text": subtitle_block,
            "highlight_color": current_highlight_color,
            "comments": "",  # Initialize an empty "Comments" column
            "speaker":speaker
        }

def interview_to_csv(input_files, output_csv_file):
    # Initialize a dictionary to store subtitles for all languages
    all_subtitles = defaultdict(lambda: defaultdict(str))

    # Iterate through each language and convert to CSV
    for language, input_file in input_files.items():
        convert_language_to_csv(language, input_file, all_subtitles)

    # Write the combined subtitles to the CSV file
    with open(output_csv_file, 'w', newline='', encoding='utf-8') as f:
        csv_writer = csv.writer(f)
        # Write the header row
        header = ["Start Time", "End Time"] + list(input_files.keys()) + ["Highlight", "Comments","Speaker"]
        csv_writer.writerow(header)
        # Write the data rows with all languages stacked
        for time_range, language_subtitles in all_subtitles.items():
            try:
                start_time, end_time = map(str.strip, time_range.split("-"))
                highlight_color = language_subtitles[list(input_files.keys())[0]]["highlight_color"]
                speaker=language_subtitles[list(input_files.keys())[0]]["speaker"]
                row = [start_time, end_time] + [clean_subtitle(language_subtitles.get(lang, {"text": ""})["text"]) for lang in input_files.keys()] + [highlight_color, "",speaker]
                csv_writer.writerow(row)
            except:
                print("broken: ", time_range)#,language_subtitles)

    print(f"Conversion completed. Data saved to {output_csv_file}")

#this is temporary until the speaker names are included directly in the transcripts
speakerDf = pd.read_csv('../02_Transcripts/Kogi/kogi-names.csv')
speakerDict= speakerDf.set_index('Interview Code').to_dict()['Interviewees']

    
# # Test
# input_files = {
#     "English": "../02_Transcripts/Wayuu/Transcripts (EN)/dunas (EN).docx",
#     "Spanish": "../02_Transcripts/Wayuu/Transcripts (ES)/dunas (ES).docx"
# }
# output_csv_file = "../04_Interview CSV/dunas.csv"

# # Convert all languages to CSV using the pipeline function
# interview_to_csv(input_files, output_csv_file)


In [8]:
#run the processing pipeline overall interviews

transcripts_dir = "../02_Transcripts/Kogi/"

def get_languages_and_interviews(transcripts_dir):
    interviews_dict = {}  # Dictionary to store interviews and their languages and file paths

    for language_folder in os.listdir(transcripts_dir):
        language_dir_path = os.path.join(transcripts_dir, language_folder)
        
        # Check if it's a valid language folder (e.g., "Transcripts (EN)")
        if os.path.isdir(language_dir_path) and language_folder.startswith("Transcripts ("):
            language = language_folder.split(" (")[1].split(")")[0]
            
            for interview_file in os.listdir(language_dir_path):
                if interview_file.endswith(".docx") and not os.path.basename(interview_file).startswith("~$"):
                    interview_name, ext = os.path.splitext(interview_file)
                    interview_code = interview_name.split("(")[0].strip().lower()  # Extract the interview code (e.g., "dunas")
                    interview_path = os.path.join(language_dir_path, interview_file)
                    
                    # Create or update the entry for this interview code
                    if interview_code not in interviews_dict:
                        interviews_dict[interview_code] = {}
                    
                    # Append language and file path to the input_files dictionary
                    interviews_dict[interview_code][language] = interview_path

    return interviews_dict    

def process_all_interviews(transcripts_dir, output_csv_dir):
    # Get the combined dictionary of languages and interviews
    interviews_dict = get_languages_and_interviews(transcripts_dir)
    
    for interview_code in interviews_dict.keys():
        print(interview_code)
        # Process the interview using the input_files and interview_code
        output_csv_file = os.path.join(output_csv_dir, f"{interview_code}.csv")
        interview_to_csv(interviews_dict[interview_code], output_csv_file)
        

# Output directory for CSV files
output_csv_dir = "../04_Interview CSV/Kogi/"

# Process all interviews in the transcripts directory
process_all_interviews(transcripts_dir, output_csv_dir)


coca
Conversion completed. Data saved to ../04_Interview CSV/Kogi/coca.csv
fuego
Conversion completed. Data saved to ../04_Interview CSV/Kogi/fuego.csv
tejiendo
Conversion completed. Data saved to ../04_Interview CSV/Kogi/tejiendo.csv
arregoces
Conversion completed. Data saved to ../04_Interview CSV/Kogi/arregoces.csv


<h3>CSVs to SRTs</h3>

In [9]:
def timecode_to_frames(text):
    if len(text.split(":"))==2:
        frames=(int(text.split(":")[0])*60+int(text.split(":")[1]))*24
    elif len(text.split(":"))==3:
        frames=(int(text.split(":")[0])*3600+int(text.split(":")[1])*60+int(text.split(":")[2]))*24
    elif len(text.split(":"))==4:
        frames=(int(text.split(":")[0])*3600+int(text.split(":")[1])*60+int(text.split(":")[2]))*24+int(text.split(":")[3])
    else:
        print(text+"timecode parse error")
    return frames

timecode_to_frames("00:31:23:02")  #test


def frames_to_srt_timecode(frames,fast=False):
    if fast==True:
        frames=frames*24/23.976 ##this is dumb, but solves the error when importing into premiere
    frames=int(frames)
    hours = frames // (3600*24)
    remaining_frames = frames % (3600*24)
    minutes=remaining_frames // (60*24)
    remaining_frames=remaining_frames % (60*24)
    seconds=remaining_frames // (24)
    remaining_frames =remaining_frames %(24)
    frames=remaining_frames

    timecode="{:02d}".format(hours)+":"+"{:02d}".format(minutes)+":"+"{:02d}".format(seconds)+","+"{:03d}".format(int(frames/24*1000))

    return timecode

frames_to_srt_timecode(45194)   #test

'00:31:23,083'

In [20]:
import os
import pandas as pd

# Input and output directories
csv_dir = "../04_Interview CSV/Kogi/"
subtitles_dir = "../05_Subtitles/Kogi/"

#fastSubs=["romelia","joaquin","magalys electrico","neko weaving","weildler inside","weildler outside","bailarinas"]

fastSubs=["coca","fuego","arregoces","tejiendo"]

# Function to convert a CSV file to SRT for multiple languages
def csv_to_srt_multiple_languages(csv_file):

    # Read the CSV file
    df = pd.read_csv(csv_file)
    
    #this should be removed, only is needed because of an error in exporting videos at 23.976 vs 24fps
    code=csv_file.split("/")[-1].split(".csv")[0].lower()
    print(code)
    fast=False
    if code in fastSubs:
        fast=True
        print(code,"is fast")

    # Get the language columns dynamically (excluding specific columns)
    exclude_columns = ["Start Time", "End Time", "Highlight", "Comments","Speaker"]
    language_columns = [col for col in df.columns if col not in exclude_columns]
    language_columns.reverse()#this is to make english last,
    
    print(language_columns)

    # Initialize a dictionary to store lines for each language
    language_srt_lines = {col: [] for col in language_columns}
    combined_srt_lines = []
    counter = 1

    # Loop through each row in the CSV
    for index, row in df.iterrows():
        # Format the timestamps in SRT format
        srt_timecode = f"{counter}\n{frames_to_srt_timecode(timecode_to_frames(row['Start Time']),fast)} --> {frames_to_srt_timecode(timecode_to_frames(row['End Time']),fast)}"

        # Append the text for each language to their respective lines
        for lang in language_columns:
            language_srt_lines[lang].append(srt_timecode)
            language_srt_lines[lang].append(f"{row[lang]}\n")

        # Append the text to the combined SRT
        combined_srt_lines.append(srt_timecode)
        combined_srt_lines.append('\n----\n'.join([str(row[lang]) for lang in language_columns]) + '\n')

        # Increment the counter
        counter += 1

    # Determine the output SRT file paths for combined and individual SRTs
    base_filename = os.path.splitext(os.path.basename(csv_file))[0]
    combined_langs = '-'.join(language_columns)
    combined_srt_filename = f"{base_filename} ({combined_langs}).srt"
    combined_srt_dir = os.path.join(subtitles_dir,"SRT Export ("+combined_langs+")")
    os.makedirs(combined_srt_dir, exist_ok=True)

    combined_srt_path= os.path.join(combined_srt_dir, combined_srt_filename)

    # Create the folder for combined SRT
    os.makedirs(subtitles_dir, exist_ok=True)

    # Write the combined SRT file
    with open(combined_srt_path, 'w', encoding='utf-8') as combined_srt_file:
        combined_srt_file.write('\n'.join(combined_srt_lines))

    # Create folders for each language and write the individual SRT files
    for lang in language_columns:
        lang_output_dir = os.path.join(subtitles_dir, f"SRT Export ({lang})")
        os.makedirs(lang_output_dir, exist_ok=True)
        lang_srt_filename = f"{base_filename} ({lang}).srt"
        lang_srt_path = os.path.join(lang_output_dir, lang_srt_filename)
        with open(lang_srt_path, 'w', encoding='utf-8') as lang_srt_file:
            lang_srt_file.write('\n'.join(language_srt_lines[lang]))

    #print(f'Combined SRT file "{combined_srt_path}" and individual SRT files have been created.')


# Process all CSV files in the input directory
for filename in os.listdir(csv_dir):
    if filename.endswith('.csv'):
        csv_file = os.path.join(csv_dir, filename)
        csv_to_srt_multiple_languages(csv_file)


fuego
fuego is fast
['ES', 'EN']
coca
coca is fast
['ES', 'EN']
tejiendo
tejiendo is fast
['ES', 'EN']
arregoces
arregoces is fast
['ES', 'EN']


<h3> Check status of txts, csv, xmls, srts </h3>


In [21]:

def getCodesPresent(folderpath,ext):
    codes = []
    for filename in os.listdir(folderpath):
        if filename.endswith(ext) and os.path.getsize(os.path.join(folderpath,filename))>1000:
            code=filename.lower().split(ext.lower())[0].strip()
            codes.append(code)
    return codes

enTexts=getCodesPresent('../02_Transcripts/Kogi/Transcripts (EN)/',' (EN).docx')
esTexts=getCodesPresent('../02_Transcripts/Kogi/Transcripts (ES)/',' (ES).docx')
xmls=getCodesPresent('../03_Interview XML/Kogi/','- final.xml')
enSubs=getCodesPresent('../05_Subtitles/Kogi/SRT Export (EN)/',' (EN).srt')
esSubs=getCodesPresent('../05_Subtitles/Kogi/SRT Export (ES)/',' (ES).srt')

def getUndefinedCount(code):
    try:
        docx_file='../02_Transcripts/Kogi/Transcripts (EN)/'+code+' (EN).docx'  
        search_string="xxx"
        count = 0
        # Load the DOCX document
        doc = docx.Document(docx_file)
        # Iterate through paragraphs and search for the string (case-insensitive)
        for paragraph in doc.paragraphs:
            if search_string.lower() in paragraph.text.lower():
                count += paragraph.text.lower().count(search_string.lower())
        return count
    except:
        return ""

lists_dict ={"EN Transcripts":enTexts,
            "ES Transcripts":esTexts,
            "EN Subs":enSubs,
            "ES Subs":esSubs,
            "XMLs":xmls}
    

# Create a DataFrame with unique codes
unique_codes = list(set(code for sublist in lists_dict.values() for code in sublist))
df = pd.DataFrame({'code': unique_codes})

# Add columns for each list with True/False values
for list_name, code_list in lists_dict.items():
    df[list_name] = df['code'].isin(code_list)

# Add column for count of XXX in document    
df["Undefined Count"]=df["code"].apply(getUndefinedCount)
    
# Set 'code' as the index
df.set_index('code', inplace=True)
 
# Print the DataFrame
print(df)


           EN Transcripts  ES Transcripts  EN Subs  ES Subs   XMLs  \
code                                                                 
fuego                True            True     True     True  False   
tejiendo             True            True     True     True  False   
coca                 True            True     True     True  False   
arregoces            True            True     True     True  False   

           Undefined Count  
code                        
fuego                   56  
tejiendo                34  
coca                    10  
arregoces               35  


<h3> Convert Script to Csv </h3>

In [22]:
def concatenate_transcript_csvs(folder_path):

    # Get a list of all CSV files in the folder
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

    # Initialize an empty DataFrame
    concatenated_df = pd.DataFrame()

    # Iterate through CSV files and concatenate them
    for csv_file in csv_files:
        interview_code = os.path.splitext(csv_file)[0]  # Extract interview code from file name
        csv_path = os.path.join(folder_path, csv_file)
        df = pd.read_csv(csv_path)

        # Add an "Interview" column with the interview code
        df['Interview'] = interview_code

        # Concatenate the dataframes
        concatenated_df = pd.concat([concatenated_df, df], ignore_index=True)
        
    concatenated_df.to_csv("../04_Interview CSV/Kogi-all.csv")

    return concatenated_df

# Example usage:
folder_path = "../04_Interview CSV/Kogi/"
concatenated_transcripts = concatenate_transcript_csvs(folder_path)
print(concatenated_transcripts)

       Start Time     End Time                                           EN  \
0     00:00:06:11  00:00:08:00                                They come out   
1     00:00:08:00  00:00:14:19                  mountains, drawing graphics   
2     00:00:14:21  00:00:16:17                                          and   
3     00:00:16:17  00:00:20:05                                   from xxxx.   
4     00:00:20:07  00:00:22:06  And is it natural for it to look like that?   
...           ...          ...                                          ...   
2663  01:02:25:08  01:02:27:01                    And so it is, we are one.   
2664  01:02:27:01  01:02:29:05      What differentiates us is the language,   
2665  01:02:29:05  01:02:31:11                    the custom, the clothing.   
2666  01:02:31:11  01:02:33:11               It's what makes the difference   
2667  01:02:33:11  01:02:36:02                          but the bet is one.   

                                       ES  Highligh

In [23]:


# Function to convert different time formats to seconds (unchanged)
def convert_time_to_seconds(time_str):
    try:
        if ':' in time_str:
            parts = time_str.split(':')
            if len(parts) == 2:
                minutes, seconds = map(int, parts)
                return minutes * 60 + seconds

        parts = time_str.split(':')
        if len(parts) == 3 or len(parts) == 4:
            hours, minutes, seconds, *milliseconds = map(int, parts)
            total_seconds = hours * 3600 + minutes * 60 + seconds
            if milliseconds:
                total_seconds += milliseconds[0] / 100
            return total_seconds
    except ValueError:
        pass

    raise ValueError(f"Unrecognized time format: {time_str}")

    # Function to convert time format to seconds
def time_to_seconds(time_str):
    time_obj = datetime.strptime(str(time_str), "%M:%S")
    return time_obj.minute * 60 + time_obj.second
    
# Function to format time in MM:SS (unchanged)
def format_time(seconds):
    minutes, seconds = divmod(seconds, 60)
    time= f"{minutes:02d}:{seconds:02d}"
    return time

def find_start_character_index(string, subsequence, cutoff=0.6):

    # Initialize the start index.
    start_index = -1

    # Iterate through the string using a sliding window approach.
    for i in range(len(string) - len(subsequence) + 1):
        window = string[i:i + len(subsequence)]

        # Use difflib's SequenceMatcher to calculate similarity.
        similarity = SequenceMatcher(None, subsequence, window).ratio()

        # If the similarity exceeds the cutoff, consider it a match.
        if similarity >= cutoff:
            start_index = i
            break

    return start_index

In [17]:
def exportNewScriptRow(matched_row,text):
    start_time_str = matched_row["Start Time"]
    start_time = convert_time_to_seconds(start_time_str)
    transcript_duration=convert_time_to_seconds(matched_row["End Time"]) - convert_time_to_seconds(matched_row["Start Time"])                    
    offset = find_start_character_index(matched_row["EN"],text[:8], cutoff=.8)
    seconds_per_letter =transcript_duration / len(matched_row["EN"])
    start_time+=offset*seconds_per_letter
    duration_seconds = len(text) * seconds_per_letter
    end_time = start_time + duration_seconds

    start_time_mmss = format_time(int(start_time))
    end_time_mmss = format_time(int(end_time))
    return start_time_mmss,end_time_mmss,duration_seconds

# Function to match script text with transcript
def match_script_with_transcript(docx_file_path, transcript_df):
    doc = docx.Document(docx_file_path)

    speaker = ""
    next_row={"EN":"temp"}
    matched_row={"EN":"temp"}
    transcript_lines = []
    for line in doc.paragraphs:
        text = str(line.text.strip())
        if any(char.isalpha() for char in text):
            match = re.match(r'\*([^, ]+)', text)
            if match:##if the line is defining a new speaker because it in eht form *Magalys
                speaker = match.group(1)
            else:

                speaker_df=transcript_df[transcript_df["Speaker"].str.contains(speaker.lower())]  

                ##first try the next line after the previously searched for line
                if text in next_row["EN"]:
                    matched_row=next_row
                    
                elif text in matched_row["EN"]:
                    matched_row=matched_row
                    
                else:
                    closest_match = get_close_matches(text, speaker_df["EN"].apply(float_to_str), n=1, cutoff=0.6)
                    if closest_match:
                        print(closest_match,text)

                        matched_row = speaker_df[speaker_df["EN"] == closest_match[0]].iloc[0]


                if closest_match or next_row["EN"]==text:

                    #this is a hacky way to get the next row but i could figure out a good index method
                    getNextRow=False
                    for index,row in speaker_df.iterrows():
                        if getNextRow:
                            next_row=row
                            getNextRow=False
                        elif row["EN"]==matched_row["EN"]:
                            getNextRow=True

                    start_time_mmss,end_time_mmss,duration_seconds=exportNewScriptRow(matched_row,text)
                    interview=matched_row["Interview"]
                else:
                    start_time_mmss=""
                    end_time_mmss=""
                    duration_seconds=""
                    interview="can't find"

                transcript_lines.append([speaker,interview, start_time_mmss, end_time_mmss, duration_seconds, text])                

    return transcript_lines

# Function to write matched transcript to CSV (unchanged)
def write_matched_transcript_to_csv(transcript_lines, output_csv_path):
    with open(output_csv_path, "w", newline="") as csvfile:
        fieldnames = ["Speaker", "Interview","Start Time", "End Time", "Duration (Seconds)", "Transcript"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for line in transcript_lines:
            speaker, interview, start_time_mmss, end_time_mmss, duration_seconds, text = line
            writer.writerow({"Speaker": speaker,"Interview":interview, "Start Time": start_time_mmss, "End Time": end_time_mmss, "Duration (Seconds)": duration_seconds, "Transcript": text})

def float_to_str(value):
    return str(value)
            
# #test
# script_path = "../01_Scripts/Wayuu/Dunas Script.docx"
# transcript_csv = pd.read_csv("../04_Interview CSV/Wayuu/dunas.csv")
# transcript_csv["Interview"]="dunas"
# output_csv_path = "../01_Scripts/Wayuu/dunas-script.csv"

# transcript_lines = match_script_with_transcript(script_path, transcript_csv)
# print(transcript_lines)
# write_matched_transcript_to_csv(transcript_lines, output_csv_path)
# print(f"CSV file with speaker, start time, end time, duration, and matched transcript created at {output_csv_path}.")


In [18]:
# #test
script_path = "../01_Scripts/Wayuu/Magalys test.docx"
concat_transcripts = concatenate_transcript_csvs("../04_Interview CSV/Wayuu/")
output_csv_path = script_path[:-5]+".csv"
transcript_lines = match_script_with_transcript(script_path, concat_transcripts)
write_matched_transcript_to_csv(transcript_lines, output_csv_path)

print("Done")



['These are,'] These are,
['This is the control that regulates the load'] This is the control that regulates the load
['Well Neko, you are entering the Macuira mountain range.\nThese are land of the Arpushana (family lineage)'] Well Neko, you are entering the Macuira mountain range.
["That's why I remain engraved in the stone of destiny\nas they say, there, at sword point"] That's why I remain engraved in the stone of destiny
['Where now'] Where now***
['They all go'] They all go***


KeyboardInterrupt: 